In [1]:
import sys
import heapq

##Função do algoritmo de Prim

In [2]:
def prim(graph, start_node):
    mst = []
    visited = set([start_node])
    edges = [
        (cost, start_node, to)
        for to, cost in graph[start_node].items()
    ]
    heapq.heapify(edges)

    while edges:
        cost, frm, to = heapq.heappop(edges)
        if to not in visited:
            visited.add(to)
            mst.append((frm, to, cost))
            for to_next, cost2 in graph[to].items():
                if to_next not in visited:
                    heapq.heappush(edges, (cost2, to, to_next))

    return mst

##Função para ler o arquivo .stp e traduzir para uma estrutura de dicionário representando um grafo

In [3]:
def read_stp(filename):
    graph = {}
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('E '):
                data = line.strip().split()
                node1 = int(data[1])
                node2 = int(data[2])
                cost = float(data[3])
                if node1 not in graph:
                    graph[node1] = {}
                if node2 not in graph:
                    graph[node2] = {}
                graph[node1][node2] = cost
                graph[node2][node1] = cost
    return graph

##Função principal que usa as anteriores para ler o arquivo de entrada, modelar o grafo e retornar a MST

In [10]:
def main():
    filename = '/content/dmxa0296.stp'
    graph = read_stp(filename)
    mst = prim(graph, 1)
    print(mst) #A Mst é retornada como uma lista de tuplas, cada tupla contém a aresta que é representada pelos seus vértices interligados e respectivo peso

In [11]:
if __name__ == "__main__":
    main()

[(1, 2, 5.0), (1, 14, 13.0), (14, 15, 5.0), (15, 27, 13.0), (27, 26, 5.0), (27, 28, 5.0), (28, 29, 5.0), (29, 30, 5.0), (30, 31, 5.0), (31, 32, 5.0), (32, 33, 5.0), (33, 34, 5.0), (27, 40, 13.0), (40, 39, 5.0), (40, 41, 5.0), (41, 42, 5.0), (42, 43, 5.0), (43, 44, 5.0), (44, 45, 5.0), (45, 46, 5.0), (46, 47, 5.0), (47, 48, 5.0), (48, 49, 5.0), (49, 50, 5.0), (50, 51, 5.0), (51, 52, 5.0), (29, 16, 13.0), (16, 17, 5.0), (17, 18, 5.0), (18, 19, 5.0), (19, 20, 5.0), (20, 21, 5.0), (16, 3, 13.0), (3, 4, 5.0), (4, 5, 5.0), (5, 6, 5.0), (6, 7, 5.0), (7, 8, 5.0), (8, 9, 5.0), (9, 10, 5.0), (10, 11, 5.0), (11, 12, 5.0), (12, 13, 5.0), (10, 22, 13.0), (22, 23, 5.0), (23, 24, 5.0), (24, 25, 5.0), (22, 35, 13.0), (35, 36, 5.0), (36, 37, 5.0), (37, 38, 5.0), (39, 53, 13.0), (53, 54, 5.0), (54, 55, 5.0), (55, 56, 5.0), (56, 57, 5.0), (57, 58, 5.0), (58, 59, 5.0), (47, 60, 13.0), (60, 61, 5.0), (61, 62, 5.0), (62, 63, 5.0), (63, 64, 5.0), (64, 65, 5.0), (53, 66, 13.0), (66, 67, 5.0), (67, 68, 5.0), (

##Exemplo de como é lido e interpretado o grafo de um arquivo de entrada

In [8]:
filename = '/content/dmxa0296.stp'
graph = read_stp(filename)

In [9]:
graph

{1: {2: 5.0, 14: 13.0},
 2: {1: 5.0, 15: 13.0},
 3: {4: 5.0, 16: 13.0},
 4: {3: 5.0, 5: 5.0, 17: 13.0},
 5: {4: 5.0, 6: 5.0, 18: 13.0},
 6: {5: 5.0, 7: 5.0, 19: 13.0},
 7: {6: 5.0, 8: 5.0, 20: 13.0},
 8: {7: 5.0, 9: 5.0, 21: 13.0},
 9: {8: 5.0, 10: 5.0},
 10: {9: 5.0, 11: 5.0, 22: 13.0},
 11: {10: 5.0, 12: 5.0, 23: 13.0},
 12: {11: 5.0, 13: 5.0, 24: 13.0},
 13: {12: 5.0, 25: 13.0},
 14: {1: 13.0, 15: 5.0},
 15: {2: 13.0, 14: 5.0, 27: 13.0},
 16: {3: 13.0, 17: 5.0, 29: 13.0},
 17: {4: 13.0, 16: 5.0, 18: 5.0, 30: 13.0},
 18: {5: 13.0, 17: 5.0, 19: 5.0, 31: 13.0},
 19: {6: 13.0, 18: 5.0, 20: 5.0, 32: 13.0},
 20: {7: 13.0, 19: 5.0, 21: 5.0},
 21: {8: 13.0, 20: 5.0, 34: 13.0},
 22: {10: 13.0, 23: 5.0, 35: 13.0},
 23: {11: 13.0, 22: 5.0, 24: 5.0, 36: 13.0},
 24: {12: 13.0, 23: 5.0, 25: 5.0, 37: 13.0},
 25: {13: 13.0, 24: 5.0, 38: 13.0},
 27: {15: 13.0, 26: 5.0, 28: 5.0, 40: 13.0},
 29: {16: 13.0, 28: 5.0, 30: 5.0, 42: 13.0},
 30: {17: 13.0, 29: 5.0, 31: 5.0, 43: 13.0},
 31: {18: 13.0, 30: 5.